In [137]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping

In [63]:
df = pd.read_csv("food_coating_recommendation.csv")

In [65]:
df.head()

,Food,Type,Moisture Content (%),pH,Texture,Perishability (1-10),Coating Type,Barrier Property,Compatibility (1-10),Shelf Life Without Coating (days),Shelf Life With Coating (days),Storage Temp (°C),Humidity (%)
0,Apple,Fruit,84,3.3,Firm,7,Carnauba wax,Moisture,9,14,30,4,90
1,Banana,Fruit,74,4.5,Soft,8,Chitosan,Antimicrobial,7,5,12,13,85
2,Strawberry,Fruit,91,3.5,Soft,9,Alginate,Antioxidant,8,3,7,2,90
3,Cucumber,Vegetable,96,5.5,Firm,6,Beeswax,Moisture,8,7,14,10,95
4,Tomato,Fruit,94,4.3,Firm,7,Zein,Oxygen,7,7,14,10,85


In [67]:
categorical_columns = df.select_dtypes(include=['object']).columns
print("Categorical columns found:", list(categorical_columns))

Categorical columns found: ['Food', 'Type', 'Texture', 'Coating Type', 'Barrier Property']


In [85]:
# Identify categorical and numerical columns
categorical_features = ['Type', 'Texture', 'Coating Type']
numerical_features = ['Moisture Content (%)', 'pH', 'Perishability (1-10)', 'Storage Temp (°C)', 'Humidity (%)']

# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features)
    ])

# Fit and transform the data
X = preprocessor.fit_transform(df)

# Get feature names after encoding
feature_names = (numerical_features + 
                 preprocessor.named_transformers_['cat']
                 .get_feature_names_out(categorical_features).tolist())

X_transformed = preprocessor.transform(df)
feature_names = (numerical_features + 
                 preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features).tolist())
df_transformed = pd.DataFrame(X_transformed, columns=feature_names)

In [87]:
df_transformed

,Moisture Content (%),pH,Perishability (1-10),Storage Temp (°C),Humidity (%),Type_Fruit,Type_Fungus,Type_Grain,Type_Meat,Type_Nut,...,Coating Type_Pullulan,Coating Type_Shellac,Coating Type_Shellac-carnauba,Coating Type_Silica-based,Coating Type_Sodium alginate,Coating Type_Soy protein,Coating Type_Whey protein,Coating Type_Xanthan gum,Coating Type_Zein,Coating Type_Zein-wax
0,0.299083,-1.620481,0.135507,-0.342876,0.347062,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.281781,-0.703779,0.772389,1.447698,-0.332601,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.705687,-1.467697,1.409271,-0.740781,0.347062,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.996119,0.060138,-0.501375,0.850840,1.026724,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.879946,-0.856563,0.135507,0.850840,-0.332601,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,-2.489062,0.289314,-0.501375,3.238271,-3.730911,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-2.430975,-0.245429,-1.138257,-0.342876,-1.012263,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,-0.165608,0.824056,1.409271,-1.138686,1.026724,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,-0.630299,0.212922,0.135507,-0.740781,0.347062,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.531428,0.442097,-1.138257,-0.342876,1.026724,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
df_transformed.to_csv('transformed_dataset.csv', index=False)

In [93]:
print(df.columns)

Index(['Food', 'Type', 'Moisture Content (%)', 'pH', 'Texture',
       'Perishability (1-10)', 'Coating Type', 'Barrier Property',
       'Compatibility (1-10)', 'Shelf Life Without Coating (days)',
       'Shelf Life With Coating (days)', 'Storage Temp (°C)', 'Humidity (%)'],
      dtype='object')


In [109]:
features = df_transformed.iloc[:, :19]  # Assuming first 19 columns are features
targets = df_transformed.iloc[:, 19:]   # Assuming remaining columns are coating types

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [185]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(19,)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(targets.shape[1], activation='sigmoid')
])

C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [187]:
def lr_schedule(epoch):
    return 0.001 * (0.1 ** int(epoch / 10))

lr_scheduler = LearningRateScheduler(lr_schedule)

model.compile(optimizer=Adam(learning_rate=0.005), loss='binary_crossentropy', metrics=['accuracy'])

In [191]:
history = model.fit(X_train_scaled, y_train, epochs = 200, batch_size=64, validation_split=0.2, verbose=1)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.7241 - loss: 0.0382 - val_accuracy: 0.0000e+00 - val_loss: 0.3082
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.6207 - loss: 0.0454 - val_accuracy: 0.0000e+00 - val_loss: 0.3098
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.7241 - loss: 0.0384 - val_accuracy: 0.0000e+00 - val_loss: 0.3120
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.5862 - loss: 0.0451 - val_accuracy: 0.0000e+00 - val_loss: 0.3148
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.5862 - loss: 0.0434 - val_accuracy: 0.0000e+00 - val_loss: 0.3171
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.7241 - loss: 0.0339 - val_accuracy: 0.0000e+00 - val_loss: 0.3196
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.6897 - loss: 0.0418 - val_accuracy: 0.0000e+00 - val_loss: 0.3220
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.7586 - loss: 0.0368 - val

In [193]:
history = model.fit(
    X_train_scaled, y_train,
    epochs=200,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.7931 - loss: 0.0177 - val_accuracy: 0.0000e+00 - val_loss: 0.4764
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.7931 - loss: 0.0253 - val_accuracy: 0.0000e+00 - val_loss: 0.4795
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.7586 - loss: 0.0183 - val_accuracy: 0.0000e+00 - val_loss: 0.4825
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.8621 - loss: 0.0109 - val_accuracy: 0.0000e+00 - val_loss: 0.4852
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.8621 - loss: 0.0159 - val_accuracy: 0.0000e+00 - val_loss: 0.4878
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.6897 - loss: 0.0259 - val_accuracy: 0.0000e+00 - val_loss: 0.4894
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.7586 - loss: 0.0152 - val_accuracy: 0.0000e+00 - val_loss: 0.4906
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.7241 - loss: 0.0262 - val

In [207]:
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.1000 - loss: 0.3664
Test accuracy: 0.1000


In [241]:
def recommend_coating(input_data):
    # Scale the input data
    input_scaled = scaler.transform(input_data)
    
    # Make predictions
    predictions = model.predict(input_scaled)
    
    # Get the top 3 recommended coatings
    top_3_indices = np.argsort(predictions[0])[-3:][::-1]
    top_3_coatings = targets.columns[top_3_indices]
    
    return top_3_coatings

In [243]:
# Example usage
sample_input = np.array([[0.29908, -1.62048, 0.13551, -0.34288, 0.34706, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]])
recommendations = recommend_coating(sample_input)
print("Top 3 recommended coatings:")
for i, coating in enumerate(recommendations, 1):
    print(f"{i}. {coating}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Top 3 recommended coatings:
1. Coating Type_Carnauba wax
2. Coating Type_Hydroxypropyl methylcellulose
3. Coating Type_Beeswax


C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [245]:
# Sample input 2: Different continuous variables, different categorical flags
sample_input_2 = np.array([[0.5, -0.8, 0.2, 0.1, 0.6, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]])

# Sample input 3: Negative values for continuous variables, multiple categorical flags
sample_input_3 = np.array([[-0.1, -1.2, -0.5, -0.7, -0.3, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]])

# Sample input 4: Extreme values for continuous variables
sample_input_4 = np.array([[2.0, -3.0, 1.5, -2.5, 1.8, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]])

# Test these inputs
for i, sample in enumerate([sample_input, sample_input_2, sample_input_3, sample_input_4], 1):
    print(f"\nSample input {i}:")
    recommendations = recommend_coating(sample)
    print("Top 3 recommended coatings:")
    for j, coating in enumerate(recommendations, 1):
        print(f"{j}. {coating}")


Sample input 1:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Top 3 recommended coatings:
1. Coating Type_Carnauba wax
2. Coating Type_Hydroxypropyl methylcellulose
3. Coating Type_Beeswax

Sample input 2:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Top 3 recommended coatings:
1. Coating Type_Carnauba wax
2. Coating Type_Whey protein
3. Coating Type_Hydroxypropyl methylcellulose

Sample input 3:


C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Top 3 recommended coatings:
1. Coating Type_Whey protein
2. Coating Type_Cellulose
3. Coating Type_Carnauba wax

Sample input 4:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Top 3 recommended coatings:
1. Coating Type_Carnauba wax
2. Coating Type_Nanocellulose
3. Coating Type_Hydroxypropyl methylcellulose


C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
